# Atelier tokenizer

Dans cet atelier nous alons constuire un tokenizer utilisant l'algorithme Byte Pair Encoding (BPE) tel que ceux utilisés
dans ChatGPT.


## Convertir une chaine de caractères en une séquence d'entiers

D'après la [documentation python](https://docs.python.org/fr/3/library/stdtypes.html#text-sequence-type-str) :

> Les chaînes sont des séquences immuables de points de code Unicode.

Chaque caractère a un numéro, une catégorie, un nom :

In [17]:
import unicodedata

text = "Bonjour 👋"

for char in text:
    print(char, ord(char), unicodedata.category(char), unicodedata.name(char))

B 66 Lu LATIN CAPITAL LETTER B
o 111 Ll LATIN SMALL LETTER O
n 110 Ll LATIN SMALL LETTER N
j 106 Ll LATIN SMALL LETTER J
o 111 Ll LATIN SMALL LETTER O
u 117 Ll LATIN SMALL LETTER U
r 114 Ll LATIN SMALL LETTER R
  32 Zs SPACE
👋 128075 So WAVING HAND SIGN


Les chaines de caractères sont ensuite encodées pour permettre la sauvegarde, la lecture, etc.

Il existe plusieurs types d'encodage. Le plus utilisé est `UTF-8`. Cet encodage représente chaque point de code 
(caractère) par une suite de 1 à 4 bytes en fonction du numéro du point de code : 


| Premier point de code | Dernier code point | Byte 1       | Byte 2   | Byte 3   | Byte 4   |
|-----------------------|--------------------|--------------|----------|----------|----------|
| U+0000                | U+007F             | **0**yyyzzzz |          |          |          |
| U+0080                | U+07FF             | **110**xxxyy | 10yyzzzz |          |          |
| U+0800                | U+FFFF             | **1110**wwww | 10xxxxyy | 10yyzzzz |          |
| U+010000              | U+10FFFF           | **11110**uvv | 10vvwwww | 10xxxxyy | 10yyzzzz |

<br/>

> **Rappel :** Un _byte_ en anglais correspond à un _octet_ en français soit 8 _bits_.
> 
> Un octet peut donc prendre 2^8 = 256 valeurs

In [22]:
text = "Bonjour 👋"

for char in text:
    for byte in char.encode("utf-8"):
        print(char, "|", byte, f"| code hexadécimal : {byte:02x}")

B | 66 | code hexadécimal : 42
o | 111 | code hexadécimal : 6f
n | 110 | code hexadécimal : 6e
j | 106 | code hexadécimal : 6a
o | 111 | code hexadécimal : 6f
u | 117 | code hexadécimal : 75
r | 114 | code hexadécimal : 72
  | 32 | code hexadécimal : 20
👋 | 240 | code hexadécimal : f0
👋 | 159 | code hexadécimal : 9f
👋 | 145 | code hexadécimal : 91
👋 | 139 | code hexadécimal : 8b


On peut directement obtenir la liste des bytes :

In [29]:
text = "Bonjour 👋"
text_ids = list(text.encode())

print(text_ids)

[66, 111, 110, 106, 111, 117, 114, 32, 240, 159, 145, 139]


## Implémentation de la tokenization Byte Pair Encoding (BPE)

[L'algorithme BPE ](https://en.wikipedia.org/wiki/Byte_pair_encoding) fonctionne en fusionnant sucessivement la paire
d'octets la plus fréquente.

Dans un premier temps, on va donc implémenter une fonction qui compte les nombres d'apparition des paires d'octet

In [33]:
text = """Une demi-heure plus tard, Harry, qui n'en croyait pas sa chance, était assis à l'arrière de la voiture des
Dursley, en compagnie de Piers et Dudley. Pour la première fois de sa vie, il allait visiter le zoo."""

text_ids = list(text.encode())

In [35]:
from collections import Counter


def count_pairs(ids: list[int]):
    return Counter(((o1, o2) for o1, o2 in zip(ids, ids[1:])))


stats = count_pairs(text_ids)
stats.most_common(3)

[((101, 32), 10), ((32, 100), 5), ((100, 101), 5)]

In [42]:
(((o1, o2), nb),) = stats.most_common(1)

(o1, o2), nb, bytes([o1, o2]).decode("utf-8")

((101, 32), 10, 'e ')

Ensuite on va écfire une fonction qui fusionne une paire en remplaçant les octets par une nouvel id.

In [50]:
def merge(ids: list[int], pair: tuple[int, int], new_id: int):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < (len(ids) - 1) and (ids[i], ids[i + 1]) == pair:
            new_ids.append(new_id)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids


merge([1, 2, 3, 1, 2, 3, 3], (2, 3), 4)

[1, 4, 1, 4, 3]

In [51]:
from tests import test_merge

test_merge(merge)